In [2]:
import matplotlib as plt
import numpy as np
import scipy
import pandas as pd
from sympy import *
import math
import qutip
import qutip as Q
import random

In [5]:
def ising_model_hamiltonian(L, J, g, periodic): # L = length of spin chain, J = overall constant, g = constant applied to field term
    # Identity and Pauli matrices for spin 1/2
    I = Q.qeye(2)
    z = Q.sigmaz()
    x = Q.sigmax()

    # create hamiltonian
    H = 0

    if periodic == 0:
        print("The string of spins is not periodic")
    
        # Interaction term
        for i in range(L - 1):
            # Tensor product betweean spin i and i+1. Other positions are identity
            term = Q.tensor([I] * i + [z] + [z] + [I] * (L - i - 2))
            H += term
            #print("The interaction term is:")
            #print(term)

        # Transverse field term
        for i in range(L):
            # Pauli x matrix at each spin in the chain while the rest are identity
            hterm = Q.tensor([I] * i + [x] + [I] * (L - i - 1))
            #print("The transverse field term is:")
            #print(hterm)
            H += g * hterm
            
    elif periodic == 1:
        print("The string of spins is periodic")
        for i in range(L - 1):
            # Tensor product betweean spin i and i+1. Other positions are identity
            term = Q.tensor([I] * i + [z] + [z] + [I] * (L - i - 2))
            H += term
            #print("The interaction term is:")
            #print(term)
            
        periodicterm = Q.tensor([I] * 0 + [z] + [I] * (L-2) + [z])
        H += periodicterm
        
        # Transverse field term
        for i in range(L):
            # Pauli x matrix at each spin in the chain while the rest are identity
            hterm = Q.tensor([I] * i + [x] + [I] * (L - i - 1))
            H += g * hterm
        
        
    H = -J * H
    print("The Hamiltonian is:")
    return H

In [19]:
H = ising_model_hamiltonian(2,1,10,0)
print(H)
E, psi = H.groundstate()
rhob = psi.ptrace([1])
print(rhob)
rhoa = psi.ptrace([0])
print(rhoa)


The string of spins is not periodic
The Hamiltonian is:
Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=CSR, isherm=True
Qobj data =
[[ -1. -10. -10.   0.]
 [-10.   1.   0. -10.]
 [-10.   0.   1. -10.]
 [  0. -10. -10.  -1.]]
Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.5        0.49937617]
 [0.49937617 0.5       ]]
Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.5        0.49937617]
 [0.49937617 0.5       ]]


In [15]:
def svd(a):
    print("the original matrix is")
    print(a)
    b = np.linalg.eig(np.matmul(np.transpose(a),a))
    #c = np.linalg.eig(np.matmul(a,np.transpose(a)))
    eigenvalues, eigenvectors = b
    
    #order eigenvalues and vectors in descending order
    idx = eigenvalues.argsort()[::-1] 
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:,idx]
    
    #set near zero terms equal to zero
    #print(eigenvalues)
    threshold1 = abs(eigenvalues) < 10**(-8)
    eigenvalues[threshold1] = 0
    
    #set near zero terms equal to zero
    #print(eigenvectors)
    threshold2 = abs(eigenvectors) < 10**(-8)
    eigenvectors[threshold2] = 0
    
    #trim zeroes from eigenvalues to make matrix sigma
    print("the eigenvalues are")
    print(eigenvalues)
    eigenvalstrimmed = np.trim_zeros(eigenvalues)
    print("the trimmed eigenvalues are")
    print(eigenvalstrimmed)
    
    #create matrix v
    print("the eigenvectors are")
    print(eigenvectors)
    v = np.transpose(eigenvectors)
    print("v is")
    print(v)
    
    #Find the singular values
    dimrow = np.count_nonzero(eigenvalstrimmed)
    dimcolumn = len(eigenvalues)
    sigma = np.zeros((dimrow,dimcolumn), dtype = float)
    svals = np.sqrt(eigenvalstrimmed)
    print("the singular values are")
    print(svals)
    
    #create matrix sigma
    np.fill_diagonal(sigma, svals)
    print("sigma is")
    print(sigma)
    print()
    
    #create matrix u
    u = np.zeros((a.shape[0], dimrow))
    for i in range(dimrow):
        u[:, i] = np.matmul(a/svals[i], eigenvectors[:, i])
    print("u is")
    print(u)
    print()
    
    #test if the SVD was correct
    originaltest = np.matmul(np.matmul(u,sigma), v)
    print("u*sigma*v gives")
    print(originaltest)
    print()
    
    #print the SVD
    print("the singular value decomposition for the matrix is")
    return u, sigma, v

In [5]:
H = ising_model_hamiltonian(2, 1, 0, 0)
print(H)
G = H.groundstate()
print(G)
gvec = np.array([[1],[0],[0],[0]])
print(gvec)

The string of spins is not periodic
The Hamiltonian is:
Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=CSR, isherm=True
Qobj data =
[[-1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0. -1.]]
(-1.0, Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[1.]
 [0.]
 [0.]
 [0.]])
[[1]
 [0]
 [0]
 [0]]


C:\Users\danny\anaconda3\Lib\site-packages\qutip\core\qobj.py:1626: UserWarning: Ground state may be degenerate.
  warnings.warn("Ground state may be degenerate.", UserWarning)


In [35]:
def create_bipartition(vec, a_size): #input vector as well as number of spins wanted in a. Remainder will be in partition b
    length = len(vec)
    indexlist = list(range(length))
    
    random.shuffle(indexlist) #create random list to ensure random selection
    
    # Select a subset of the list
    a_index = indexlist[:a_size]
    
    # The remaining b list
    b_index = indexlist[a_size:]

    print('The index for a spins is')
    print(a_index)
    print('The index for b spins is')
    print(b_index)

In [36]:
def matrix_from_vector(vec, a_index, b_index):
    a_list = []
    for i in a_index:
        #print(i)
        a_list.append(vec[i][0])
        
    b_list = []
    for i in b_index:
        #print(i)
        b_list.append(vec[i][0])
    
    print('The list of a spins is')
    print(a_list)
    print('the list of b spins is')
    print(b_list)

In [37]:
vec = np.array([[1],[2],[3],[4]])
create_bipartition(vec, 2)
matrix_from_vector(vec, [2,3], [1,0])

The index for a spins is
[2, 3]
The index for b spins is
[0, 1]
The list of a spins is
[3, 4]
the list of b spins is
[2, 1]


array([[0, 0],
       [0, 0]])